In [5]:
import sys
sys.path.append('..')
import os, pickle
import torch
import numpy as np
from torch.utils.data import DataLoader
from src.networks import GraphCVAE
from src.data import ConditionDataset, ReactionDataset
from src.trainer import VAEClassTrainer
from src.feature import parse_feature
from src.utils import squared_error, cosin_similarity, cyclical_kld_annealing
from sklearn.metrics import accuracy_score, f1_score

In [33]:
#DS_c = ConditionDataset(target_feat_type='active_composit',
#                        precursor_feat_type='active_composit')
#DS_c.from_file(data_path='../data/unique_reaction.pkl.gz')
#DS_c.to('cpu')

DS_r = ReactionDataset(target_feat_type='active_composit',
                        precursor_feat_type='active_composit')
DS_r.from_file(data_path='../data/unique_reaction.pkl.gz')
DS_r.to('cpu')

root_t = '/home/jhyang/WORKSPACES/MODELS/isyn/VAE_test'

#model_fcnn = ClassCVAE(42, 16, 8, 200)
#model_gnn  = GraphClassCVAE()
#model.load(root_t + '/cls/fcnn_active_composit_batch_0128_mdim_32_128_4', 'model')

In [47]:
years = np.array([d.year for d in DS_r])
test_mask = years > 2018
train_mask = years < 2017
valid_mask = ~train_mask & ~test_mask

from torch.utils.data.sampler import SubsetRandomSampler
train_dl = DataLoader(DS_r, batch_size=256, sampler=SubsetRandomSampler(np.where(train_mask)[0]), collate_fn=DS_r.cfn)

In [46]:
model_gnn = GraphCVAE(input_dim = DS_r.num_precursor_feat, 
                      edge_dim = DS_r.num_target_feat,
                      latent_dim = 32, 
                      output_dim = DS_r.num_class,
                      condition_dim = DS_r.num_metal_feat,
                      graph = 'conv',
                      encoder_hidden_dim = 64, 
                      encoder_hidden_layers = 4, 
                      decoder_hidden_dim = 64, 
                      decoder_hidden_layers = 4,
)

In [48]:
tr = VAEClassTrainer(model_gnn, 1e-3, device='cpu')

In [55]:
feat = next(iter(train_dl))[0]

In [64]:
model_gnn(**feat)[1].

tensor(nan, grad_fn=<MulBackward0>)

In [43]:
feat = DS_r.cfn(DS_r[:256])[0]
out = model_gnn(**feat)
#for k,v in DS_r.cfn([DS_r[0], DS_r[1]])[0].items():
#    print(k,v)

In [69]:
import gzip, pickle
with gzip.open('../data/unique_precursor.pkl.gz','rb') as f:
    prec = pickle.load(f)
prec[0].keys()

dict_keys(['years', 'precursor_comp'])